In [49]:
import pandas as pd

In [50]:
census=pd.read_csv('../Tensorflow-Bootcamp-master/02-TensorFlow-Basics/census_data.csv')

In [51]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [52]:
census['income_bracket'].unique()

array([' <=50K', ' >50K'], dtype=object)

In [53]:
def label_fix(label):
    if label=='<=50k':
        return 0
    else:
        return 1

In [54]:
census['income_bracket']=census['income_bracket'].apply(label_fix)

In [55]:
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,1
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,1
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,1
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,1
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,1


In [56]:
from sklearn.model_selection import train_test_split

In [57]:
x_data=census.drop('income_bracket',axis=1)

In [58]:
y_labels=census['income_bracket']

In [59]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y_labels, test_size=0.33, random_state=101)

In [60]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [61]:
import tensorflow as tf

In [62]:
census.columns

Index(['age', 'workclass', 'education', 'education_num', 'marital_status',
       'occupation', 'relationship', 'race', 'gender', 'capital_gain',
       'capital_loss', 'hours_per_week', 'native_country', 'income_bracket'],
      dtype='object')

In [63]:
gender=tf.feature_column.categorical_column_with_vocabulary_list('gender',['Female','Male'])

In [64]:
occupation=tf.feature_column.categorical_column_with_hash_bucket('occupation',hash_bucket_size=1000)

In [65]:
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size=1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size=1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size=1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size=1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size=1000)

In [66]:
age = tf.feature_column.numeric_column("age")
education_num = tf.feature_column.numeric_column("education_num")
capital_gain = tf.feature_column.numeric_column("capital_gain")
capital_loss = tf.feature_column.numeric_column("capital_loss")
hours_per_week = tf.feature_column.numeric_column("hours_per_week")

In [67]:
feat_cols=[gender,occupation,marital_status,relationship,education,workclass,native_country,age,education_num
          ,capital_gain,capital_loss,hours_per_week]

In [77]:
input_func=tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,
                                              num_epochs=None,shuffle=True)

In [78]:
# model=tf.estimator.LinearClassifier(feature_columns=feat_cols)
dnn_model = tf.estimator.DNNClassifier(hidden_units=[10,10,10],feature_columns=feat_cols,n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Rahul\\AppData\\Local\\Temp\\tmp325dca2f', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000015DFB7C66A0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [70]:
dnn_model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Calling model_fn.


ValueError: Items of feature_columns must be a _DenseColumn. You can wrap a categorical column with an embedding_column or indicator_column. Given: _VocabularyListCategoricalColumn(key='gender', vocabulary_list=('Female', 'Male'), dtype=tf.string, default_value=-1, num_oov_buckets=0)

In [73]:
gender = tf.feature_column.embedding_column(gender,dimension=2)
occupation= tf.feature_column.embedding_column(occupation,dimension=1000)
marital_status= tf.feature_column.embedding_column(marital_status,dimension=1000)
relationship= tf.feature_column.embedding_column(relationship,dimension=1000)
education = tf.feature_column.embedding_column(education,dimension=1000) 
workclass= tf.feature_column.embedding_column(workclass,dimension=1000)
native_country= tf.feature_column.embedding_column(native_country,dimension=1000)

In [74]:
feat_cols=[gender,occupation,marital_status,relationship,education,workclass,native_country,age,education_num
          ,capital_gain,capital_loss,hours_per_week]

In [79]:
dnn_model.train(input_fn=input_func,steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Rahul\AppData\Local\Temp\tmp325dca2f\model.ckpt.
INFO:tensorflow:loss = 542.3083, step = 1
INFO:tensorflow:global_step/sec: 37.2687
INFO:tensorflow:loss = 0.00011078892, step = 101 (2.683 sec)
INFO:tensorflow:global_step/sec: 47.7784
INFO:tensorflow:loss = 1.7124328e-05, step = 201 (2.095 sec)
INFO:tensorflow:global_step/sec: 48.3019
INFO:tensorflow:loss = 2.1406637e-05, step = 301 (2.069 sec)
INFO:tensorflow:global_step/sec: 50.0496
INFO:tensorflow:loss = 7.503096e-05, step = 401 (2.000 sec)
INFO:tensorflow:global_step/sec: 48.345
INFO:tensorflow:loss = 0.0003127202, step = 501 (2.070 sec)
INFO:tensorflow:global_step/sec: 48.4269
INFO:tensorflow:loss = 1.183034e-05, step = 601 (2.061 sec)
INFO:tens

INFO:tensorflow:loss = 4.5865877e-07, step = 7701 (2.042 sec)
INFO:tensorflow:global_step/sec: 49.7263
INFO:tensorflow:loss = 2.4446868e-07, step = 7801 (2.011 sec)
INFO:tensorflow:global_step/sec: 48.412
INFO:tensorflow:loss = 2.4599576e-05, step = 7901 (2.067 sec)
INFO:tensorflow:global_step/sec: 48.6669
INFO:tensorflow:loss = 1.9553888e-06, step = 8001 (2.059 sec)
INFO:tensorflow:global_step/sec: 48.905
INFO:tensorflow:loss = 8.0774015e-07, step = 8101 (2.042 sec)
INFO:tensorflow:global_step/sec: 50.1966
INFO:tensorflow:loss = 3.815116e-07, step = 8201 (1.990 sec)
INFO:tensorflow:global_step/sec: 48.7734
INFO:tensorflow:loss = 1.294377e-06, step = 8301 (2.054 sec)
INFO:tensorflow:global_step/sec: 48.1047
INFO:tensorflow:loss = 6.195672e-08, step = 8401 (2.075 sec)
INFO:tensorflow:global_step/sec: 49.4117
INFO:tensorflow:loss = 2.542519e-06, step = 8501 (2.028 sec)
INFO:tensorflow:global_step/sec: 49.4025
INFO:tensorflow:loss = 9.262462e-06, step = 8601 (2.020 sec)
INFO:tensorflow:gl

In [80]:
pred_fn=tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [81]:
pred_gen=model.predict(input_fn=pred_fn)

In [82]:
predictions=list(pred_gen)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Rahul\AppData\Local\Temp\tmpa_d1pcpe\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [83]:
predictions

[{'logits': array([21.914713], dtype=float32),
  'logistic': array([1.], dtype=float32),
  'probabilities': array([3.0378136e-10, 1.0000000e+00], dtype=float32),
  'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([13.091112], dtype=float32),
  'logistic': array([0.999998], dtype=float32),
  'probabilities': array([2.0634852e-06, 9.9999797e-01], dtype=float32),
  'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([19.653387], dtype=float32),
  'logistic': array([1.], dtype=float32),
  'probabilities': array([2.915026e-09, 1.000000e+00], dtype=float32),
  'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object)},
 {'logits': array([10.447095], dtype=float32),
  'logistic': array([0.9999709], dtype=float32),
  'probabilities': array([2.903165e-05, 9.999709e-01], dtype=float32),
  'class_ids': array([1], dtype=int64),
  'classes': array([b'1'], dtype=object)},
 {'logits':

In [84]:
final_preds=[pred['class_ids'][0] for pred in predictions]

In [85]:
final_preds

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [86]:
eval_input_func = tf.estimator.inputs.pandas_input_fn(
      x=X_test,
      y=y_test,
      batch_size=10,
      num_epochs=1,
      shuffle=False)

In [87]:
model.evaluate(input_fn=eval_input_func)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-29-18:58:36
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\Rahul\AppData\Local\Temp\tmpa_d1pcpe\model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-29-18:58:41
INFO:tensorflow:Saving dict for global step 10000: accuracy = 1.0, accuracy_baseline = 1.0, auc = 0.0, auc_precision_recall = 1.0, average_loss = 1.9337294e-06, global_step = 10000, label/mean = 1.0, loss = 1.9330098e-05, precision = 1.0, prediction/mean = 0.999999, recall = 1.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 10000: C:\Users\Rahul\AppData\Local\Temp\tmpa_d1pcpe\model.ckpt-10000


{'accuracy': 1.0,
 'accuracy_baseline': 1.0,
 'auc': 0.0,
 'auc_precision_recall': 1.0,
 'average_loss': 1.9337294e-06,
 'label/mean': 1.0,
 'loss': 1.9330098e-05,
 'precision': 1.0,
 'prediction/mean': 0.999999,
 'recall': 1.0,
 'global_step': 10000}

In [88]:
from sklearn.metrics import classification_report

In [89]:
print(classification_report(y_test,final_preds))

             precision    recall  f1-score   support

          1       1.00      1.00      1.00     10746

avg / total       1.00      1.00      1.00     10746

